In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import gzip
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import time
import math

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)
print('ok')

ok


In [2]:
cnt=0
for l in readGz("data/train.json.gz"):
    print(l)
    cnt+=1
    if cnt>=5:
        break

{'rating': 5.0, 'businessID': 'B408037852', 'reviewText': "This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': 'Mar 24, 2013', 'categories': ["Women's Clothing Store", 'Fashion Accessories Store', 'Shoe Store'], 'reviewHash': 'R471510664', 'unixReviewTime': 1364143460}
{'rating': 5.0, 'businessID': 'B410583770', 'reviewText': "Quaint, stylish atmosphere in a small place right off South Congress. The food and service were great. It's pricey, but the happy hour deals are considerable. Great for a date!", 'userID': 'U637588426', 'reviewTime': 'Sep 21, 2011', 'categories': ['Italian Restaurant', 'European Restaurant'], 'reviewHash': 'R954562590', 'unixReviewTime': 1316616175}
{'rating': 4.0, 'businessID': 'B423621081', 'reviewText': "I'm a vegetarian, but every so often I want a hotdog with lots of toppings.  And a tall can of beer.  F

In [3]:
users=[]
bussiness=[]
ratings=[]
for l in readGz("data/train.json.gz"):
    u=l['userID']
    b=l['businessID']
    r=float(l['rating'])
    users.append(u)
    bussiness.append(b)
    ratings.append(r)
    
user_set=set(users)
buss_set=set(bussiness)

Nu=len(user_set)  # 18793
Nb=len(buss_set)  # 21321

user_index={}
buss_index={}

for (i,u) in enumerate(user_set):
    user_index[u]=i
for (i,b) in enumerate(buss_set):
    buss_index[b]=i
print(Nu,Nb)
print('ok')

18793 21321
ok


In [4]:
num_train=199999

train_set=[]
val_set=[]

buss_u={}
user_b={}

n_train=0
n_val=0

total_rating_train=0

index=list(range(len(users)))
random.shuffle(index)

for i in index:
    u=user_index[users[i]]
    b=buss_index[bussiness[i]]
    r=ratings[i]
    if i<num_train:
        total_rating_train+=r
        train_set.append((u,b,r))
        if u not in buss_u:
            buss_u[u]=[(b,r)]
        else:
            buss_u[u].append((b,r))
        if b not in user_b:
            user_b[b]=[(u,r)]
        else:
            user_b[b].append((u,r))
        n_train+=1
    else:
        val_set.append((u,b,r))
        n_val+=1

print('ok')


ok


In [5]:
# lamb=1.0
# max_iter=500
# print_every=10

# alpha=0
# beta_u=np.zeros(Nu)
# beta_b=np.zeros(Nb)

# for it in range(max_iter):
#     alpha=0
#     for (u,b,r) in train_set:
#         alpha+=(r-beta_u[u]-beta_b[b])/n_train
#     for (u,b_r_list) in buss_u.items():
#         beta_u[u]=0
#         for (b,r) in b_r_list:
#             beta_u[u]+=(r-alpha-beta_b[b])/(lamb+len(b_r_list))
#     for (b,u_r_list) in user_b.items():
#         beta_b[b]=0
#         for (u,r) in u_r_list:
#             beta_b[b]+=(r-alpha-beta_u[u])/(lamb+len(u_r_list))
    
#     loss=0.0
#     RMSE=0.0
#     for (u,b,r) in train_set:
#         loss+=(alpha+beta_u[u]+beta_b[b]-r)**2
#         RMSE+=(alpha+beta_u[u]+beta_b[b]-r)**2
#     loss+=lamb*(np.sum(np.square(beta_u))+np.sum(np.square(beta_b)))
#     RMSE/=n_train
#     RMSE=math.sqrt(RMSE)
     
#     if it==0 or (it+1)%print_every==0:
#         print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'iter %d, loss=%.10f, RMSE=%.10f' %(it+1,loss,RMSE))


In [6]:
# lamb=4.0
# max_iter=10000
# print_every=50
# # leaning_rate=1e-5
# weight_scale=1

# alpha=total_rating_train/n_train
# # beta_u=np.zeros(Nu)
# # beta_b=np.zeros(Nb)
# beta_u=np.random.randn(Nu)*weight_scale
# beta_b=np.random.randn(Nb)*weight_scale

# K=1
# gamma_u=np.random.randn(Nu,K)*weight_scale
# gamma_b=np.random.randn(Nb,K)*weight_scale

# min_RMSE=None
# best_parameters=None

# def adam(x, dx, config=None):
#     if config is None: config = {}
#     config.setdefault('learning_rate', 1e-2)
#     config.setdefault('beta1', 0.9)
#     config.setdefault('beta2', 0.999)
#     config.setdefault('epsilon', 1e-8)
#     config.setdefault('m', np.zeros_like(x))
#     config.setdefault('v', np.zeros_like(x))
#     config.setdefault('t', 0)
#     next_x = None
#     config['t']+=1
#     config['m']=config['beta1']*config['m']+(1-config['beta1'])*dx
#     config['v']=config['beta2']*config['v']+(1-config['beta2'])*(dx**2)
#     m=config['m']/(1-config['beta1']**config['t'])
#     v=config['v']/(1-config['beta2']**config['t'])
#     next_x=x-config['learning_rate']*m/(np.sqrt(v)+config['epsilon'])
#     return next_x, config

# config_alpha=None
# config_beta_u=None
# config_beta_b=None
# config_gamma_u=None
# config_gamma_b=None

# for it in range(max_iter):
    
#     dalpha=0.0
#     dbeta_u=np.zeros(Nu)
#     dbeta_b=np.zeros(Nb)
#     dgamma_u=np.zeros((Nu,K))
#     dgamma_b=np.zeros((Nb,K))
    
#     loss=0.0
#     RMSE=0.0
#     for (u,b,r) in train_set:
#         temp=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-r)
#         loss+=0.5*temp**2
#         RMSE+=temp**2
#         dalpha+=temp
#         dbeta_u[u]+=temp
#         dbeta_b[b]+=temp
#         dgamma_u[u,:]+=temp*gamma_b[b,:]
#         dgamma_b[b,:]+=temp*gamma_u[u,:]
        
#     loss+=0.5*lamb*(np.sum(np.square(beta_u))+np.sum(np.square(beta_b))+np.sum(np.square(gamma_u))+np.sum(np.square(gamma_b)))
#     dbeta_u+=lamb*beta_u
#     dbeta_b+=lamb*beta_b
#     dgamma_u+=lamb*gamma_u
#     dgamma_b+=lamb*gamma_b
    
# #     alpha-=leaning_rate*dalpha
# #     beta_u-=leaning_rate*dbeta_u
# #     beta_b-=leaning_rate*dbeta_b
# #     gamma_u-=leaning_rate*dgamma_u
# #     gamma_b-=leaning_rate*dgamma_b

#     alpha,config_alpha=adam(alpha,dalpha,config_alpha)
#     beta_u,config_beta_u=adam(beta_u,dbeta_u,config_beta_u)
#     beta_b,config_beta_b=adam(beta_b,dbeta_b,config_beta_b)
#     gamma_u,config_gamma_u=adam(gamma_u,dgamma_u,config_gamma_u)
#     gamma_b,config_gamma_b=adam(gamma_b,dgamma_b,config_gamma_b)
    
#     RMSE/=n_train
#     RMSE=math.sqrt(RMSE)
    
#     RMSE_val=0.0
#     for (u,b,real_r) in val_set:
#         RMSE_val+=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-real_r)**2
#     RMSE_val/=len(val_set)
#     RMSE_val=math.sqrt(RMSE_val)
    
#     if it==0 or (it+1)%print_every==0:
#         print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
#               'iter %d, loss=%.10f, RMSE=%.10f RMSE_val=%.10f' %(it+1,loss,RMSE,RMSE_val))

# # alpha,beta_u,beta_b,gamma_u,gamma_b=best_parameters

In [7]:
# lamb1=4.0
# lamb2=10.0
# max_iter=700
# print_every=10
# weight_scale=1
# leanring_rate=5e-3

# alpha=0
# beta_u=np.zeros(Nu)
# beta_b=np.zeros(Nb)

# K=10
# gamma_u=np.random.rand(Nu,K)*2.0-1.0
# gamma_b=np.random.rand(Nb,K)*2.0-1.0
# # gamma_u=np.zeros((Nu,K))
# # gamma_b=np.zeros((Nb,K))

# for it in range(max_iter):
    
#     alpha=0
#     for (u,b,r) in train_set:
#         alpha+=(r-beta_u[u]-beta_b[b]-np.inner(gamma_u[u,:],gamma_b[b,:]))/n_train
#     for (u,b_r_list) in buss_u.items():
#         beta_u[u]=0
#         for (b,r) in b_r_list:
#             beta_u[u]+=(r-alpha-beta_b[b]-np.inner(gamma_u[u,:],gamma_b[b,:]))/(lamb1+len(b_r_list))
#     for (b,u_r_list) in user_b.items():
#         beta_b[b]=0
#         for (u,r) in u_r_list:
#             beta_b[b]+=(r-alpha-beta_u[u]-np.inner(gamma_u[u,:],gamma_b[b,:]))/(lamb1+len(u_r_list))
    
#     dgamma_u=np.zeros_like(gamma_u)
#     dgamma_b=np.zeros_like(gamma_b)

#     loss=0.0
#     RMSE=0.0
#     for (u,b,r) in train_set:
#         temp=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-r)
#         loss+=0.5*temp**2
#         RMSE+=temp**2
#         dgamma_u[u,:]+=temp*gamma_b[b,:]
#         dgamma_b[b,:]+=temp*gamma_u[u,:]

#     loss+=0.5*(lamb1*(np.sum(np.square(beta_u))+np.sum(np.square(beta_b)))+lamb2*(np.sum(np.square(gamma_u))+np.sum(np.square(gamma_b))))
#     dgamma_u+=lamb2*gamma_u
#     dgamma_b+=lamb2*gamma_b

#     gamma_u-=leanring_rate*dgamma_u
#     gamma_b-=leanring_rate*dgamma_b
    
#     RMSE/=n_train
#     RMSE=math.sqrt(RMSE)
    
#     RMSE_val=0.0
#     for (u,b,real_r) in val_set:
#         RMSE_val+=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-real_r)**2
#     RMSE_val/=len(val_set)
#     RMSE_val=math.sqrt(RMSE_val)
    
#     if it==0 or (it+1)%print_every==0:
#         print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
#               'iter %d, loss=%.10f, RMSE=%.10f RMSE_val=%.10f' %(it+1,loss,RMSE,RMSE_val))

In [8]:
lamb=4.0
max_iter=300
print_every=50

alpha=0
beta_u=np.zeros(Nu)
beta_b=np.zeros(Nb)

for it in range(max_iter):

    alpha=0
    for (u,b,r) in train_set:
        alpha+=(r-beta_u[u]-beta_b[b])/n_train
    for (u,b_r_list) in buss_u.items():
        beta_u[u]=0
        for (b,r) in b_r_list:
            beta_u[u]+=(r-alpha-beta_b[b])/(lamb+len(b_r_list))
    for (b,u_r_list) in user_b.items():
        beta_b[b]=0
        for (u,r) in u_r_list:
            beta_b[b]+=(r-alpha-beta_u[u])/(lamb+len(u_r_list))

    loss=0.0
    RMSE=0.0
    for (u,b,r) in train_set:
        temp=(alpha+beta_u[u]+beta_b[b]-r)
        loss+=0.5*temp**2
        RMSE+=temp**2

    loss+=0.5*lamb*(np.sum(np.square(beta_u))+np.sum(np.square(beta_b)))

    RMSE/=n_train
    RMSE=math.sqrt(RMSE)

    if it==0 or (it+1)%print_every==0:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
                'iter %d, loss=%.10f, RMSE=%.10f' %(it+1,loss,RMSE))

2017-11-20 09:57:29 iter 1, loss=50621.6151630884, RMSE=0.6645356605
2017-11-20 09:58:20 iter 50, loss=49702.8318668271, RMSE=0.6610365569
2017-11-20 09:59:11 iter 100, loss=49702.8318668018, RMSE=0.6610365586
2017-11-20 10:00:02 iter 150, loss=49702.8318668019, RMSE=0.6610365586
2017-11-20 10:00:53 iter 200, loss=49702.8318668019, RMSE=0.6610365586
2017-11-20 10:01:44 iter 250, loss=49702.8318668019, RMSE=0.6610365586
2017-11-20 10:02:35 iter 300, loss=49702.8318668019, RMSE=0.6610365586


In [9]:
max_iter=2000
print_every=50
K=1
lamb=10.0
leanring_rate=5e-3
weight_scale=1
gamma_u=np.random.randn(Nu,K)*weight_scale
gamma_b=np.random.randn(Nb,K)*weight_scale

for it in range(max_iter):
    dgamma_u=np.zeros((Nu,K))
    dgamma_b=np.zeros((Nb,K))

    loss=0.0
    RMSE=0.0
    for (u,b,r) in train_set:
        temp=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-r)
        loss+=0.5*temp**2
        RMSE+=temp**2
        dgamma_u[u,:]+=temp*gamma_b[b,:]
        dgamma_b[b,:]+=temp*gamma_u[u,:]

    loss+=0.5*lamb*(np.sum(np.square(gamma_u))+np.sum(np.square(gamma_b)))
    dgamma_u+=lamb*gamma_u
    dgamma_b+=lamb*gamma_b

    gamma_u-=leanring_rate*dgamma_u
    gamma_b-=leanring_rate*dgamma_b
    
    RMSE/=n_train
    RMSE=math.sqrt(RMSE)
    
    RMSE_val=0.0
    for (u,b,real_r) in val_set:
        RMSE_val+=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-real_r)**2
    RMSE_val/=len(val_set)
    RMSE_val=math.sqrt(RMSE_val)
    
    if it==0 or (it+1)%print_every==0:
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
              'iter %d, loss=%.10f, RMSE=%.10f RMSE_val=%.10f' %(it+1,loss,RMSE,RMSE_val))

2017-11-20 10:02:38 iter 1, loss=341873.1605297659, RMSE=1.1923073574 RMSE_val=0.8497129830
2017-11-20 10:04:21 iter 50, loss=44679.8387243222, RMSE=0.6592198210 RMSE_val=0.6035000703
2017-11-20 10:06:07 iter 100, loss=43684.4441187169, RMSE=0.6598539660 RMSE_val=0.6005044108
2017-11-20 10:07:53 iter 150, loss=43597.2378665499, RMSE=0.6580283578 RMSE_val=0.6005466219
2017-11-20 10:09:38 iter 200, loss=43571.5305448508, RMSE=0.6569009285 RMSE_val=0.6005549831
2017-11-20 10:11:22 iter 250, loss=43553.8774268596, RMSE=0.6561808198 RMSE_val=0.6005559431
2017-11-20 10:13:05 iter 300, loss=43545.4300717998, RMSE=0.6557267576 RMSE_val=0.6005562952
2017-11-20 10:14:48 iter 350, loss=43542.9717898313, RMSE=0.6555423043 RMSE_val=0.6005565020
2017-11-20 10:16:39 iter 400, loss=43541.3457223667, RMSE=0.6554132892 RMSE_val=0.6005566344
2017-11-20 10:18:22 iter 450, loss=43540.1827264264, RMSE=0.6553098447 RMSE_val=0.6005567324
2017-11-20 10:20:07 iter 500, loss=43539.3283739911, RMSE=0.6552275426 R

In [10]:
print(alpha,beta_u,beta_b,gamma_u,gamma_b)
# alpha,beta_u,beta_b,gamma_u,gamma_b=best_parameters
# print(alpha,beta_u,beta_b,gamma_u,gamma_b)
# print(alpha+beta_u[1]+beta_b[1]+np.inner(gamma_u[1],gamma_b[1]))

4.22500325204 [-0.01642814 -0.01298683 -0.73022023 ..., -0.14971447  0.03783178
 -0.25373654] [ 0.28633772  0.19729145 -0.09843808 ...,  0.24859883  0.06354608
  0.03736198] [[  4.60454399e-05]
 [  3.17774084e-03]
 [  3.70272687e-02]
 ..., 
 [  6.82267819e-03]
 [ -6.93752275e-07]
 [  2.99407620e-03]] [[  6.19780138e-02]
 [ -4.76389735e-06]
 [ -1.89520625e-02]
 ..., 
 [  9.86468829e-02]
 [  2.07113999e-05]
 [  1.85442400e-03]]


In [11]:
# (alpha,beta_u,beta_b,gamma_u,gamma_b)=best_parameters

RMSE=0.0

for (u,b,real_r) in val_set:
#     RMSE+=(alpha+beta_u[u]+beta_b[b]-real_r)**2
    RMSE+=(alpha+beta_u[u]+beta_b[b]+np.inner(gamma_u[u,:],gamma_b[b,:])-real_r)**2

RMSE/=len(val_set)
RMSE=math.sqrt(RMSE)
print('RMSE =',RMSE)

RMSE = 0.6005572565267956


In [12]:
test_file=open('data/pairs_Rating.txt')
fo=open('data/pairs_Rating_out.txt','w')
fo.write('userID-businessID,prediction\n')
c1=0
c2=0
c3=0
for line in test_file:
    if line == 'userID-businessID,prediction\n':
        continue
    [u,b]=line.strip().split('-')
    if u not in user_index:
        predict=total_rating_train/n_train
        c1+=1
    elif b not in buss_index:
        predict=alpha+beta_u[user_index[u]]
        c2+=1
    else:  
        ui=user_index[u]
        bi=buss_index[b]
#         predict=alpha+beta_u[ui]+beta_b[bi]
        predict=alpha+beta_u[ui]+beta_b[bi]+np.inner(gamma_u[ui,:],gamma_b[bi,:])
        c3+=1
    fo.write('%s-%s,%f\n' %(u,b,predict))
fo.close()
print(c1,c2,c3)

11 13 19976


In [13]:
# a=np.array([1,2,3,4])
# b=np.array([2,3,4,5])
# y1=a*(np.sum(b)*b)
# print(y1)
# y2=np.array([0,0,0,0])
# for i in range(4):
#     y2[i]=np.inner(a,b)*b[i]
# print(y2)